Create an assistant that uses any LLM and should use pydantic, when we ask about any product it should give you three information product_name, product_details and tentative_price in USD.

Use chat prompt template

In [ ]:
# Import the 'os' module, which allows interaction with the operating system (e.g., accessing environment variables)
import os

# Import the 'load_dotenv' function from the 'dotenv' package
# This function is used to load environment variables from a .env file into the system environment
from dotenv import load_dotenv

# Call the 'load_dotenv' function to read variables from a .env file (if it exists in the current directory)
# and add them to the environment (os.environ)
load_dotenv()

True

In [5]:
# Set the environment variable "OPENAI_API_KEY" using the value fetched from the existing environment (likely loaded from .env)
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# Set the environment variable "LANGCHAIN_API_KEY" from the value already present in the environment
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")

# Set the environment variable "LANGCHAIN_PROJECT" from the environment value (could refer to a project name or ID)
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")

# Set the environment variable "GROQ_API_KEY" using the value loaded from environment (.env or system)
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

# Enable LangChain tracing v2 by explicitly setting this environment variable to the string "true"
os.environ["LANGCHAIN_TRACING_V2"] = "true"


In [28]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field

parser = JsonOutputParser(pydantic_object=BaseModel)

class ProductDetails(BaseModel):
    product_name: str = Field(description="The name of the product")
    product_details: str = Field(description="Detailed description of the product")
    tentative_price: float = Field(description="Estimated price of the product in USD")

prompt = ChatPromptTemplate(
    [
        ("system", "You are a helpful e-commerce assistant that provides the product name, product details and tentative price of the product in USD.The output should be in JSON format with the following keys: 'product_name' with data type as str, 'product_details' data type as str, and 'tentative_price' data type as float."),
        ("user", "{text}"),
    ]
)
prompt

ChatPromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="You are a helpful e-commerce assistant that provides the product name, product details and tentative price of the product in USD.The output should be in JSON format with the following keys: 'product_name' with data type as str, 'product_details' data type as str, and 'tentative_price' data type as float."), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='{text}'), additional_kwargs={})])

In [29]:
llm_model = ChatGroq(model="gemma2-9b-it",temperature=0.0, max_tokens=5000)

In [30]:
chain = prompt | llm_model | parser


In [31]:
output = chain.invoke({"text":"Provide me info on the iphone 15 pro max."})
print(output)

{'product_name': 'iPhone 15 Pro Max', 'product_details': '6.7-inch Super Retina XDR display, A17 Pro chip, 48MP main camera, LiDAR scanner, up to 8GB RAM, 256GB to 1TB storage', 'tentative_price': 1099.0}
